In [20]:
import random
from tqdm import tqdm
import math
import numpy as np
from submodlib import FacilityLocationFunction, GraphCutFunction, DisparityMinFunction, DisparitySumFunction
import pickle
import time
import os
import timeit
import pandas as pd

In [2]:
def stochastic_greedy(D, k, objFL, ϵ=1e-2, n=10, test=False):
    """Samples n subsets using stochastic-greedy.
    Args:
        D: number of training example
        k: The subset size.
        objFL: submodular function for conditional gain calculation
        ϵ: The error tolerance.
        n: The number of subsets to sample.

    Returns:
        A list of n subsets.
    """
    
    # Initialize empty subsets
    S = [set() for _ in range(n)]
    # Set random subset size for the stochastic-greedy algorithm
    s = int(D * math.log(1 / ϵ) / k)
    for i in tqdm(range(n)):
        for j in range(k):
            # Sample a random subset by sampling s elements from D \ Si
            R = random.choices(list(set(range(D)) - S[i]), k=s)
            # Use map to calculate marginal gains for all values in R with the custom set
            marginal_gains = list(map(lambda r: objFL.marginalGain(S[i], r), R))
            max_index = np.argmax(marginal_gains)
            max_r = R[max_index]
            S[i].add(max_r)
            if test:
                print(R)
                print(marginal_gains)
                print(max_index, max_r)
                return S
    return S

In [3]:
def SGE_optimised(D, k, objFL, ϵ=1e-2, n=10, test=False):
    """Samples n subsets using stochastic-greedy.
    Args:
        D: number of training example
        k: The subset size.
        objFL: submodular function for conditional gain calculation
        ϵ: The error tolerance.
        n: The number of subsets to sample.

    Returns:
        A list of n subsets.
    """
    
    # Initialize empty subsets
    S = [set() for _ in range(n)]
    # Set random subset size for the stochastic-greedy algorithm
    s = int(D * math.log(1 / ϵ) / k)
    for i in tqdm(range(n)):
        for j in range(k):
            # Sample a random subset by sampling s elements from D \ Si
            R = random.choices(list(set(range(D)) - S[i]), k=s)
            # Use map to calculate marginal gains for all values in R with the custom set
            marginal_gains = list(map(lambda r: objFL.marginalGainWithMemoization(S[i], r), R))
            max_index = np.argmax(marginal_gains)
            max_r = R[max_index]
            objFL.updateMemoization(S[i], max_r)
            S[i].add(max_r)
            if test:
                print(R)
                print(marginal_gains)
                print(max_index, max_r)
                return S
        objFL.clearMemoization()
    return S


In [4]:
def WRE(D, k, obj, n=10):
    """Samples n subsets using weighted random exploration.
    Args:
        D: number of training example
        k: The subset size.
        objFL: submodular function for conditional gain calculation
        n: The number of subsets to sample.
    Returns:
        A list of n subsets.
    """

    A = set()
    G = np.zeros(D)
    for i in range(D):
        R = list(set(range(D))-A)
        marginal_gains = list(map(lambda r: obj.marginalGain(A, r), R))
        max_index = np.argmax(marginal_gains)
        max_r = R[max_index]
        G[max_r] = marginal_gains[max_index]
        A.add(max_r)
    ts = 1+G+0.5*G**2
    ts = ts/np.sum(ts)
    S = [set() for _ in range(n)]
    for i in range(n):
        S[i] = np.random.choice(len(ts), size=k, p=ts, replace=False)
    return S

In [5]:
def WRE_optimised(D, k, obj, n=10):
    """Samples n subsets using weighted random exploration.
    Args:
        D: number of training example
        k: The subset size.
        obj: submodular function for conditional gain calculation
        n: The number of subsets to sample.
    Returns:
        A list of n subsets.
    """

    A = set()
    G = np.zeros(D)
    for i in tqdm(range(D)):
        R = list(set(range(D))-A)
        # if i%100==99:
            # print(len(R))
        marginal_gains = list(map(lambda r: obj.marginalGainWithMemoization(A, r), R))
        max_index = np.argmax(marginal_gains)
        max_r = R[max_index]
        G[max_r] = marginal_gains[max_index]
        obj.updateMemoization(A, max_r)
        A.add(max_r)
    ts = 1+G+0.5*G**2
    ts = ts/np.sum(ts)
    S = [set() for _ in range(n)]
    for i in tqdm(range(n)):
        S[i] = np.random.choice(len(ts), size=k, p=ts, replace=False)
    return S

In [6]:
def WRE_max(D, k, obj, n=10):
    """Samples n subsets using weighted random exploration.
    Args:
        D: number of training example
        k: The subset size.
        obj: submodular function for conditional gain calculation
        n: The number of subsets to sample.
    Returns:
        A list of n subsets.
    """

    A = set()
    G = np.zeros(D)
    for i in tqdm(range(D)):
        R = list(set(range(D))-A)
        marginal_gains = list(map(lambda r: obj.marginalGainWithMemoization(A, r), R))
        max_index = np.argmax(marginal_gains)
        max_r = R[max_index]
        G[max_r] = marginal_gains[max_index]
        obj.updateMemoization(A, max_r)
        A.add(max_r)
    
    G = G -G.min()
    
    ts = 1+G+0.5*G**2
    ts = ts/np.sum(ts)
    S = [set() for _ in range(n)]
    for i in tqdm(range(n)):
        S[i] = np.random.choice(len(ts), size=k, p=ts, replace=False)
    return S

In [7]:
import numpy as np
from concurrent.futures import ThreadPoolExecutor

def euclidean_distance(data, i, j):
    return np.linalg.norm(data[i] - data[j])

def euclidean_kernel_threaded(data, n_threads=8):
    n_samples = data.shape[0]
    kernel = np.zeros((n_samples, n_samples))

    with ThreadPoolExecutor(max_workers=n_threads) as executor:
        tasks = []
        for i in range(n_samples):
            for j in range(i + 1, n_samples):
                tasks.append(executor.submit(euclidean_distance, data, i, j))

        for i, j, future in zip(*[iter(tasks)] * 3):
            distance = future.result()
            kernel[i, j] = 1 / (distance**2 + 1e-8)
            kernel[j, i] = kernel[i, j]

    return kernel

In [8]:
# kernel = euclidean_kernel_threaded(data)
# print(kernel.shape)

In [16]:
subset_size_fraction = 0.3
dataset = "cifar10"
metric = "euclidean"

In [17]:
with open(f"{dataset}/dataframe.pkl", "rb") as f:
    df = pickle.load(f)

In [26]:
with open(f"./cifar10-dino-cls/dataframe-train.pkl", "rb") as f:
    df = pickle.load(f)

In [27]:
groups = df.groupby('Label')
dataframes = [group for _, group in groups]

In [28]:
print(df.columns)

Index(['Index', 'Label', 'Features'], dtype='object')


# Generate milo subsets

In [29]:
from tqdm import tqdm

In [30]:
lambdaVal = 0.4

funcs = ["facility-location", "disparity-min",  "disparity-sum", "graph-cut"]
fracs = [0.3, 0.15, 0.1, 0.05, 0.5]

run_time = {}
for func in funcs:
    run_time[func] = {}
    for subset_size_fraction in fracs:
        time_to_get_subsets = 0
        num_sets = 100
        for i, df in tqdm(enumerate(dataframes)):
            features = df["Features"].to_numpy()
            
            if func=="facility-location":
                obj = FacilityLocationFunction(n=features.shape[0], data=features, separate_rep=False, mode="dense", metric="cosine")
            elif func=="disparity-min":
                obj = DisparityMinFunction(n=features.shape[0], data=features, mode="dense", metric="cosine")
            elif func=="disparity-sum":
                obj = DisparitySumFunction(n=features.shape[0], data=features, mode="dense", metric="cosine")
            elif func=="graph-cut":
                obj = GraphCutFunction(n=features.shape[0], data=features, mode="dense", metric="cosine", lambdaVal=lambdaVal)
            else:
                raise Exception("Sorry, no submodlib function defined")

#             setup = """
# subset_size_fraction = {}  # Pass the value into setup
# from __main__ import SGE_optimised, features, obj, num_sets
# """.format(subset_size_fraction)
            
            # stmt = "SGE_optimised(features.shape[0], int(features.shape[0] * subset_size_fraction), obj, n=num_sets)"
            # time_to_get_subsets += timeit.timeit(stmt, setup, number=1)
            
            start_time = time.time()
            S = SGE_optimised(features.shape[0], int(features.shape[0]*subset_size_fraction), obj, n=num_sets)
            time_to_get_subsets += time.time() - start_time

            set_indexes = []
            for j in range(num_sets):
                set_indexes.append(set(df.iloc[list(S[j])].Index.tolist()))
            
            # path = f"./{dataset}/SGE-{metric}-2/{func}/class-data-{subset_size_fraction}-{num_sets}"
            path = f"./cifar10-dino-cls/SGE-{metric}-2/{func}/class-data-{subset_size_fraction}-{num_sets}"
            if not os.path.exists(path):
                os.makedirs(path)

            with open(f"{path}/class_{i}.pkl", "wb") as f:
                pickle.dump(set_indexes, f)
                
            # list_of_class_wise_subsets.append(set_indexes)

        if func=="graph-cut":
                run_time[func][f"{fracs}-{lambdaVal}"] = time_to_get_subsets 
        else:
            run_time[func][f"{fracs}"] = time_to_get_subsets 
            
        # print("Submodlib funciton: %s, time:--- %s seconds ---" % (func, time_to_get_subsets))

 38%|███▊      | 38/100 [02:46<04:32,  4.39s/it]
0it [03:05, ?it/s]


KeyboardInterrupt: 

### other

In [13]:
# func = "facility-location"
# func = "disparity-min"
# func = "disparity-sum"
# func = "graph-cut"
lambdaVal = 0.5

num_runs = 1
funcs = ["facility-location", "disparity-min",  "disparity-sum", "graph-cut"]
fracs = [0.05, 0.1, 0.15, 0.3, 0.5]

run_time = {}
for j in range(num_runs):
    run_time[j] = {}
    for func in funcs:
        run_time[j][func] = {}
        for subset_size_fraction in fracs:
            time_to_get_subsets = 0
            num_sets = 20
            for i, df in enumerate(dataframes):
                features = df["Features"].to_numpy()
                
                if func=="facility-location":
                    obj = FacilityLocationFunction(n=features.shape[0], data=features, separate_rep=False, mode="dense", metric="cosine")
                elif func=="disparity-min":
                    obj = DisparityMinFunction(n=features.shape[0], data=features, mode="dense", metric="cosine")
                elif func=="disparity-sum":
                    obj = DisparitySumFunction(n=features.shape[0], data=features, mode="dense", metric="cosine")
                elif func=="graph-cut":
                    obj = GraphCutFunction(n=features.shape[0], data=features, mode="dense", metric="cosine", lambdaVal=lambdaVal)
                else:
                    raise Exception("Sorry, no submodlib function defined")

                start_time = time.time()
                S = SGE_optimised(features.shape[0], int(features.shape[0]*subset_size_fraction), obj, n=num_sets)
                time_to_get_subsets += time.time() - start_time

                set_indexes = []
                for j in range(num_sets):
                    set_indexes.append(set(df.iloc[list(S[j])].Index.tolist()))
                
                path = f"./{dataset}/SGE-{metric}/{func}/class-data-{subset_size_fraction}"
                if not os.path.exists(path):
                    os.makedirs(path)

                with open(f"{path}/class_{i}.pkl", "wb") as f:
                    pickle.dump(set_indexes, f)
                    
                # list_of_class_wise_subsets.append(set_indexes)

            # if func=="graph-cut":
            #         run_time[j][func][f"{fracs}-{lambdaVal}"] = time_to_get_subsets 
            # else:
            #     run_time[j][func][f"{fracs}"] = time_to_get_subsets 
                
            # print("Submodlib funciton: %s, time:--- %s seconds ---" % (func, time_to_get_subsets))

KeyError: 9

In [ ]:
import pickle

with open("cifar10/SGE-euclidean-2/facility-location/class-data-0.3/class_0.pkl", "rb") as f:
    data = pickle.load(f)

In [ ]:
for d in data:
    print(d)

{32784, 24594, 49171, 8217, 32796, 49181, 16421, 32808, 16427, 48, 41012, 32829, 49221, 78, 41044, 49236, 8285, 32869, 8299, 32878, 8307, 116, 16499, 24698, 16507, 8321, 8326, 16522, 8334, 8340, 152, 8345, 49314, 16552, 49325, 32945, 8387, 24778, 32972, 32984, 49373, 41185, 24807, 49390, 8431, 8433, 33012, 41205, 49401, 41211, 41214, 41216, 49413, 49417, 271, 49430, 24856, 16667, 295, 24877, 41266, 41278, 49471, 16704, 24898, 33092, 41296, 16724, 41306, 8547, 49510, 16743, 41329, 41335, 16761, 41340, 24958, 8581, 16778, 49566, 426, 8618, 25003, 16815, 41400, 8633, 33219, 49604, 16842, 41424, 49619, 487, 8683, 25077, 41463, 49671, 33289, 33292, 8718, 16918, 535, 537, 539, 25115, 41514, 8748, 8753, 25137, 41522, 564, 41524, 25149, 8770, 41540, 8783, 597, 8791, 16985, 604, 605, 609, 17000, 8823, 49799, 25225, 652, 8849, 25240, 41626, 667, 17071, 49859, 41668, 710, 17104, 49874, 8921, 49890, 744, 8937, 41709, 8943, 17137, 49910, 17147, 25352, 25353, 8977, 17170, 17175, 17179, 41757, 49966,

## Facility Location Function

In [14]:
time_to_get_subsets = 0
list_of_class_wise_subsets = []
num_sets = 20
for i, df in enumerate(dataframes):
    features = df["Features"].to_numpy()
    objFL = FacilityLocationFunction(n=features.shape[0], data=features, separate_rep=False, mode="dense", metric="cosine")
    start_time = time.time()
    S = SGE_optimised(features.shape[0], int(features.shape[0]*subset_size_fraction), objFL, n=num_sets)
    time_to_get_subsets += time.time() - start_time
    set_indexes = []
    for j in range(num_sets):
        set_indexes.append(set(df.iloc[list(S[0])].Index.tolist()))
    path = f"./{dataset}/SGE/facility-location/class-data-{subset_size_fraction}"
    if not os.path.exists(path):
        os.makedirs(path)
    with open(f"{path}/class_{i}.pkl", "wb") as f:
        pickle.dump(set_indexes, f)
    list_of_class_wise_subsets.append(set_indexes)
print("--- %s seconds ---" % (time_to_get_subsets))

 20%|██        | 4/20 [02:51<11:25, 42.86s/it]


KeyboardInterrupt: 

## Disparity-Min Fucntion

In [ ]:
time_to_get_subsets = 0
list_of_class_wise_subsets = []
num_sets = 20
for i, df in enumerate(dataframes):
    features = df["Features"].to_numpy()
    objDM = DisparityMinFunction(n=features.shape[0], data=features, mode="dense", metric="cosine")
    start_time = time.time()
    S = SGE_optimised(features.shape[0], int(features.shape[0]*subset_size_fraction), objDM, n=num_sets)
    time_to_get_subsets += time.time() - start_time
    set_indexes = []
    for j in range(num_sets):
        set_indexes.append(set(df.iloc[list(S[0])].Index.tolist()))
        
    with open(f"./{dataset}/SGE/disparity-min/class-data-{subset_size_fraction}/class_{i}.pkl", "wb") as f:
        pickle.dump(set_indexes, f)
    list_of_class_wise_subsets.append(set_indexes)
print("--- %s seconds ---" % (time_to_get_subsets))

## Graph Cut Function

In [23]:
lambdaVal = 0.5
objGC = GraphCutFunction(n=features.shape[0], data=features, mode="dense", metric="cosine", lambdaVal=lambdaVal)

In [ ]:
time_to_get_subsets = 0
list_of_class_wise_subsets = []
num_sets = 20
for i, df in enumerate(dataframes):
    features = df["Features"].to_numpy()
    objGC = GraphCutFunction(n=features.shape[0], data=features, mode="dense", metric="cosine", lambdaVal=lambdaVal)
    start_time = time.time()
    S = SGE_optimised(features.shape[0], int(features.shape[0]*subset_size_fraction), objGC, n=num_sets)
    time_to_get_subsets += time.time() - start_time
    set_indexes = []
    for j in range(num_sets):
        set_indexes.append(set(df.iloc[list(S[0])].Index.tolist()))
    with open(f"./{dataset}/SGE/graph-cut/class-data-{subset_size_fraction}/class_{i}.pkl", "wb") as f:
        pickle.dump(set_indexes, f)
    list_of_class_wise_subsets.append(set_indexes)
print("--- %s seconds ---" % (time_to_get_subsets))

## Disparity Sum Function

In [24]:
objDS = DisparitySumFunction(n=features.shape[0], data=features, mode="dense", metric="cosine")

In [ ]:
time_to_get_subsets = 0
list_of_class_wise_subsets = []
num_sets = 20
for i, df in enumerate(dataframes):
    features = df["Features"].to_numpy()
    objDS = DisparitySumFunction(n=features.shape[0], data=features, mode="dense", metric="cosine")
    start_time = time.time()
    S = SGE_optimised(features.shape[0], int(features.shape[0]*subset_size_fraction), objDS, n=num_sets)
    time_to_get_subsets += time.time() - start_time
    set_indexes = []
    for j in range(num_sets):
        set_indexes.append(set(df.iloc[list(S[0])].Index.tolist()))
    with open(f"./{dataset}/SGE/disparity-sum/class-data-{subset_size_fraction}/class_{i}.pkl", "wb") as f:
        pickle.dump(set_indexes, f)
    list_of_class_wise_subsets.append(set_indexes)
print("--- %s seconds ---" % (time_to_get_subsets))

In [ ]:
with open("./class-data/class_0.pkl", "rb") as f:
    data = pickle.load(f)

In [15]:
features = dataframes[0]["Features"].to_numpy()

In [16]:
print(type(features[0]))

<class 'numpy.ndarray'>


In [20]:
objDM = DisparityMinFunction(n=features.shape[0], data=features, mode="dense", metric="cosine")

In [12]:
S = SGE_optimised(features.shape[0], features.shape[0]//10, objFL)

100%|██████████| 10/10 [00:23<00:00,  2.39s/it]


In [16]:
print(type(dataframes[0]["Index"]))
print(S[0])

<class 'pandas.core.series.Series'>
{2056, 2063, 2068, 2076, 2081, 4130, 4131, 4132, 41, 4137, 2092, 46, 2095, 51, 53, 4151, 58, 2112, 2115, 2121, 74, 4171, 80, 83, 4181, 2134, 2133, 4186, 2142, 96, 4194, 108, 4205, 2157, 4208, 2162, 2163, 2169, 4219, 2171, 4221, 4223, 2177, 4227, 4229, 137, 4246, 2200, 153, 2206, 4255, 4257, 2215, 2216, 4266, 2220, 2222, 182, 2241, 2242, 2251, 4302, 206, 209, 215, 2267, 231, 2281, 234, 4336, 242, 246, 249, 4349, 2305, 4354, 2316, 2318, 4367, 4375, 281, 2330, 2339, 2347, 2351, 307, 4414, 4419, 4422, 329, 2382, 334, 4432, 2383, 337, 2387, 4436, 2408, 4458, 364, 365, 2420, 2422, 2423, 376, 4475, 2428, 386, 391, 2440, 4491, 405, 4506, 4507, 4509, 420, 4522, 2477, 2479, 433, 4540, 4545, 2505, 457, 2508, 2519, 4572, 481, 2534, 487, 4589, 4590, 506, 4611, 2565, 4618, 523, 2576, 4625, 2578, 4629, 538, 4635, 4642, 4643, 552, 4649, 4656, 2609, 4657, 564, 2612, 4663, 4671, 589, 591, 4688, 592, 2639, 2644, 597, 4693, 4695, 2646, 2652, 4702, 2660, 614, 616, 2668, 

In [19]:
index_values = dataframes[0].iloc[list(S[0])].Index
print(type(index_values))


<class 'pandas.core.series.Series'>


TypeError: __init__() missing 1 required positional argument: 'lambdaVal'

In [ ]:
import time
start_time = time.time()
S = objFL.maximize(features.shape[0]-1, optimizer='NaiveGreedy', stopIfZeroGain=False, stopIfNegativeGain=False, epsilon=0.1, verbose=False, show_progress=True, costs=None, costSensitiveGreedy=False)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
S = objFL.maximize(features.shape[0]-1, show_progress=True)

In [ ]:
print("Why is this running slow")

In [ ]:
S = WRE_optimised(features.shape[0], features.shape[0]//10, objFL, n=20)

In [ ]:
S = stochastic_greedy(features.shape[0], features.shape[0]//10, objFL)

In [ ]:
S1 = stochastic_greedy(features.shape[0], features.shape[0]//10, objFL, test=True)

In [ ]:
features2 = dataframes[1]["Features"].to_numpy()
objFL2 = FacilityLocationFunction(n=features2.shape[0], data=features2, separate_rep=False, mode="dense", metric="cosine")

In [ ]:
S2 = SGE_optimised(features2.shape[0], features2.shape[0]//10, objFL2)

In [ ]:
S3 = WRE_optimised(features2.shape[0], features2.shape[0]//10, objFL2)

In [ ]:
import itertools
import numpy as np

def calculate_iou(set1, set2):
    intersection = set1.intersection(set2)
    union = set1.union(set2)
    iou = len(intersection) / len(union)
    return iou


def calculate_iou_matrix(sets_of_integers):
    num_sets = len(sets_of_integers)
    iou_matrix = np.zeros((num_sets, num_sets))

    for i, j in itertools.combinations(range(num_sets), 2):
        set1 = sets_of_integers[i]
        set2 = sets_of_integers[j]
        iou = calculate_iou(set1, set2)
        iou_matrix[i, j] = iou
        iou_matrix[j, i] = iou  # Fill the symmetric part

    return iou_matrix


In [ ]:
calculate_iou(S2[0], S2[0])

In [ ]:
iou_matrix = calculate_iou_matrix(S2)

In [ ]:
print(iou_matrix)